In [135]:
import pandas as pd
import numpy as np
df = pd.read_csv('model_data.csv').drop(columns = ['Unnamed: 0','f_highrate','f_lowrate'])

In [136]:
df.columns

Index(['id', 'user_id', 'shift_id', 'prev_CW/SA_rate', 'status',
       'U_approve2now', 'prev_CW x SA_rate', 'type_RN', 'type_LVN+LPN',
       'type_STNA', 'segmentName_d', 'net_pay', 'target', 'sa_create',
       'Start_Time', 'areaName_Austin', 'areaName_Cincinnati', 'areaName_DFW',
       'areaName_Houston', 'areaName_Northeast Ohio', 'count_prev_SA',
       'count_prev_CW', 'type_CNA', 'reliability_score'],
      dtype='object')

# Data Prepration

### Slice df by the end of this week, for predcition output

In [137]:
from datetime import date, timedelta
tmrrw = date.today() + timedelta(days=1)

end_of_week = str(tmrrw.year) + '-' + str(tmrrw.month) + '-' + str(tmrrw.day+1)

# convert to datetime for conditonal selection
df['Start_Time'] = pd.to_datetime(df['Start_Time'])
# sort by start time -> for slicing
df = df.sort_values(by = 'Start_Time') 
# record as realdata
realdata = df[df['Start_Time'].apply(lambda x: x > pd.to_datetime(end_of_week))]
# record predction output rows, don't include it in tran test validation
realdata_len = realdata.shape[0]
# only keep status = confirmed
realdata = realdata[realdata['status'] == 'confirmed']

###  <font color = green> Validation set: 1000 recently records

In [138]:
# slice, dont include realdata
validation = df[-1000-realdata_len : -realdata_len]

y_valid = validation['target']
x_valid = validation.drop(['id','user_id', 'shift_id', 'status', 'sa_create', 'Start_Time', 'target'], axis = 1)

y_valid.value_counts()

0    918
1     82
Name: target, dtype: int64

### Train test: main dataset - validation set

In [139]:
df = df[:-1000-realdata_len] # slice 

In [140]:
df = df.dropna()

In [141]:
X = df.drop(['id','user_id', 'shift_id', 'status', 'target', 'sa_create', 'Start_Time'], axis = 1)
y = df['target']

# set test, train
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [142]:
# make experienment dataset
df1 = df[(df['areaName_Northeast Ohio'] == 1) & (df['type_STNA']==1)]
X_exp = df1.drop(['id','user_id', 'shift_id', 'status', 'target', 'sa_create', 'Start_Time'], axis = 1)
y_exp = df1['target']

In [143]:
df['target'].value_counts()

0    72452
1     5973
Name: target, dtype: int64

# Logistic Regression 1  

In [144]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

# assign less punlishment for classifying 0 as 1 -> find more 1's
# weights = {0:1, 1:10}
# class_weight = 'balanced': automatically adjust weights inversely proportional to class frequencies in the input data
logit = LogisticRegression(solver = 'lbfgs', max_iter=100000, class_weight = 'balanced')
logit.fit(X_train,y_train)

LogisticRegression(class_weight='balanced', max_iter=100000)

### Train Test result

In [145]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = logit.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print('\n')
print(classification_report(y_test, y_pred))

[[14798  6957]
 [  615  1158]]


              precision    recall  f1-score   support

           0       0.96      0.68      0.80     21755
           1       0.14      0.65      0.23      1773

    accuracy                           0.68     23528
   macro avg       0.55      0.67      0.52     23528
weighted avg       0.90      0.68      0.75     23528



In [146]:
from sklearn.metrics import roc_curve
from numpy import sqrt
from numpy import argmax

# predict probabilities
yhat = logit.predict_proba(X_exp)
# keep probabilities for the positive outcome only
yhat = yhat[:, 1]

# calculate roc curves
fpr, tpr, thresholds = roc_curve(y_exp,yhat)

# calculate the g-mean for each threshold
gmeans = sqrt(tpr * (1-fpr))

# locate the index of the largest g-mean
ix = argmax(gmeans)

lower_limiter = thresholds[ix]
print('Best Threshold=%f' % (lower_limiter))

Best Threshold=0.549319


In [147]:
# search thresholds for imbalanced classification
from numpy import arange
from numpy import argmax
from sklearn.datasets import make_classification
from sklearn.metrics import f1_score
# apply threshold to positive probabilities to create labels
def to_labels(pos_probs, threshold):
    return (pos_probs >= threshold).astype('int')

# predict probabilities
yhat = logit.predict_proba(X_exp)
# keep probabilities for the positive outcome only
probs = yhat[:, 1]
# define thresholds
thresholds = arange(0, 1, 0.001)
# evaluate each threshold
scores = [f1_score(y_exp, to_labels(probs, t)) for t in thresholds]
# get best threshold
ix = argmax(scores)

higher_limiter = thresholds[ix]

print('Best threshold=%.3f' % (higher_limiter))

Best threshold=0.657


In [148]:
# from cf_matrix import make_confusion_matrix
# labels = ['True Neg','False Pos','False Neg','True Pos']
# categories = ['Zero', 'One']
# make_confusion_matrix(confusion_matrix(y_test, y_pred), 
#                       group_names=labels,
#                       categories=categories, 
#                       cmap='Blues')

In [149]:
# logit summary
import statsmodels.api as sm
smlogit = sm.Logit(y_train,X_train).fit()
smlogit.summary()

Optimization terminated successfully.
         Current function value: 0.260807
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 target   No. Observations:                54897
Model:                          Logit   Df Residuals:                    54880
Method:                           MLE   Df Model:                           16
Date:                Mon, 07 Jun 2021   Pseudo R-squ.:                 0.03460
Time:                        16:46:03   Log-Likelihood:                -14318.
converged:                       True   LL-Null:                       -14831.
Covariance Type:            nonrobust   LLR p-value:                2.720e-208
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
prev_CW/SA_rate             2.3308      0.201     11.584      0.000       1.936       2.725
U_approve2now              -0.0469      0.003    -17.046      0.000      -0.052      -0.042
prev_CW x SA_rate           0.0001   2.32e-05      4.565      0.000    6.05e-05       0.000
type_RN                    -2.5319      0.143    -17.734      0.000      -2.812      -2.252
type_LVN+LPN               -1.2149      0.088    -13.783      0.000      -1.388      -1.042
type_STNA                  -0.7624      0.100     -7.587      0.000      -0.959      -0.565
segmentName_d              -0.7510      0.057    -13.153      0.000      -0.863      -0.639
net_pay                     0.0248      0.003      8.943      0.000       0.019       0.030
areaName_Austin            -0.9878      0.086    -11.515      0.000      -1.156      -0.820
areaName_Cincinnati        -0.4282      0.239     -1.788      0.074      -0.898       0.041
areaName_DFW               -0.9977      0.066    -15.065      0.000      -1.128      -0.868
areaName_Houston           -1.2084      0.059    -20.343      0.000      -1.325      -1.092
areaName_Northeast Ohio    -0.7928      0.095     -8.335      0.000      -0.979      -0.606
count_prev_SA              -0.0009      0.001     -1.115      0.265      -0.003       0.001
count_prev_CW               0.0441      0.013      3.277      0.001       0.018       0.070
type_CNA                   -0.6598      0.057    -11.510      0.000      -0.772      -0.547
reliability_score          -0.0003      0.000     -0.914      0.361      -0.001       0.000
===========================================================================================
"""

### Overfitting? No

In [150]:
y_pred = logit.predict(X_train)

print(confusion_matrix(y_train, y_pred))
print('\n')
print(classification_report(y_train, y_pred))

[[34462 16235]
 [ 1459  2741]]


              precision    recall  f1-score   support

           0       0.96      0.68      0.80     50697
           1       0.14      0.65      0.24      4200

    accuracy                           0.68     54897
   macro avg       0.55      0.67      0.52     54897
weighted avg       0.90      0.68      0.75     54897



In [151]:
# test threshold
limiter = higher_limiter

y_prob = list(logit.predict_proba(X_train)[:,1])
y_pred = []
count =0
for prob in y_prob:
    if prob >= limiter:
        y_pred.append(1)
        count+=1
    else:
        y_pred.append(0)

print(confusion_matrix(y_train, y_pred))
print('\n')
print(classification_report(y_train, y_pred))

[[45939  4758]
 [ 2849  1351]]


              precision    recall  f1-score   support

           0       0.94      0.91      0.92     50697
           1       0.22      0.32      0.26      4200

    accuracy                           0.86     54897
   macro avg       0.58      0.61      0.59     54897
weighted avg       0.89      0.86      0.87     54897



### <font color = green> Validation result

In [152]:
# test threshold
limiter = higher_limiter

y_prob = list(logit.predict_proba(x_valid)[:,1])
y_pred = []
count = 0
for prob in y_prob:
    if prob >= limiter:
        y_pred.append(1)
        count+=1
    else:
        y_pred.append(0)

print(confusion_matrix(y_valid, y_pred))
print('\n')
print(classification_report(y_valid, y_pred))

[[813 105]
 [ 42  40]]


              precision    recall  f1-score   support

           0       0.95      0.89      0.92       918
           1       0.28      0.49      0.35        82

    accuracy                           0.85      1000
   macro avg       0.61      0.69      0.63      1000
weighted avg       0.90      0.85      0.87      1000



In [153]:
from sklearn.metrics import recall_score

label_coverage = y_pred.count(1)/len(y_pred)
UCW_coverage = recall_score(y_valid, y_pred)

print('The limiter we adopt is %.3f' % (limiter))
print('By covering %.3f labeled as high probability of UCW, we have prepared for %.3f of real UCW' 
      % (label_coverage,UCW_coverage))

The limiter we adopt is 0.657
By covering 0.145 labeled as high probability of UCW, we have prepared for 0.488 of real UCW


# Fit real data in this model

In [154]:
# set input
real_X = realdata.drop(['id','user_id', 'shift_id', 'status', 'target', 'sa_create', 'Start_Time'], axis = 1)

In [155]:
# concat predicted prob with data
realdata['prob'] = list(logit.predict_proba(real_X)[:,1])

In [156]:
# record when this prediction is ran
from datetime import date
time = str(date.today().year) + '-' + str(date.today().month) + '-' + str(date.today().day)
limiter = round(limiter,2)
realdata[['id', 'Start_Time', 'prob']].head(5)

,id,Start_Time,prob
80717,199102,2021-06-09 05:00:00,0.636987
80537,198788,2021-06-09 05:00:00,0.555280
80732,199129,2021-06-09 05:00:00,0.490924
80549,198805,2021-06-09 05:00:00,0.572149
79762,197341,2021-06-09 05:00:00,0.490933


In [157]:
high_prob = realdata[['id', 'Start_Time', 'prob']][realdata['prob'] > limiter]

## Append newly processed data to prediction data

In [158]:
import pandas as pd
# specify connection to database
import psycopg2
connection = psycopg2.connect(
    host="nursedash-prod.cuzi2kducsnv.us-east-1.rds.amazonaws.com",
    database="nursedash",
    user="external_analyst",
    password="vWHYpF9CNtC9KWBG7sZ5JvX9")

### <font color = green> all time to chicago time, No withdrawn info

In [159]:
df = pd.read_sql_query("""

SELECT  sa.id, sa.user_id, sa.shift_id, f.id AS facility_id, sa."withdrawnInfo" -> 'initiator' as withdrawnInfo_value,
sa."status", sa."prevStatus", sa."distance", s."facility_id", "s"."description" AS "shift_description",
"s"."assigned_nurse_id", s."net_pay", "s"."unit" AS "s_unit",s."type",
"s"."qualifications" AS "s_qualifications", "s"."breakTime" AS "s_breakTime", sa."withdrawnInfo",
"f"."name" AS "facility_name","f"."short_name" AS "f_short_name", f."segmentName", f."areaName",
timezone('America/Chicago', s."createdAt") as s_create,
timezone('America/Chicago', sa."createdAt") as sa_create,
timezone('America/Chicago', u."approvedAt") as u_approve,
timezone('America/Chicago', u."createdAt") as u_create,
timezone('America/Chicago', sa."statusUpdatedAt") as sa_statusUpdate,
timezone('America/Chicago', timezone('UTC', s.start_time)) AS "Start_Time" 
FROM shifts s
INNER JOIN shift_applications sa ON s.id = sa.shift_id
INNER JOIN facilities f ON s.facility_id = f.id
INNER JOIN users u ON sa.user_id = u.id

""", con = connection)

In [160]:
df.columns

Index(['id', 'user_id', 'shift_id', 'facility_id', 'withdrawninfo_value',
       'status', 'prevStatus', 'distance', 'facility_id', 'shift_description',
       'assigned_nurse_id', 'net_pay', 's_unit', 'type', 's_qualifications',
       's_breakTime', 'withdrawnInfo', 'facility_name', 'f_short_name',
       'segmentName', 'areaName', 's_create', 'sa_create', 'u_approve',
       'u_create', 'sa_statusupdate', 'Start_Time'],
      dtype='object')

In [161]:
def get_part_of_day(hour):
    return (
        "morning" if 4 < hour <= 12
        else
        "afternoon" if 12 < hour <= 17
        else
        "evening/night" if 18 < hour <= 22
        else
        "overnight"

    )

df['Start_time_of_the_day'] = df.apply(lambda row: get_part_of_day(row['Start_Time'].hour), axis =1)

# combine the prediction file with real data

In [162]:
# read the prediction file
prediction = realdata[['id', 'Start_Time', 'prob']]
validation = prediction.merge(df, on = 'id', how = 'left')

In [163]:
from datetime import date

today = date.today()

# convert to datetime for conditonal selection
validation['Start_Time_x'] = pd.to_datetime(validation['Start_Time_x'])

# only select date part of the time
validation['Start_Time_x'] = validation.apply(lambda row: str(row['Start_Time_x'].date()), axis = 1)

In [164]:
# rename start time
validation = validation.rename(columns={"Start_Time_x": "Start_Time"})

# limit our result to what we want as validation file
validation = validation[['id','prob','Start_Time','Start_time_of_the_day','status','type','prevStatus','areaName','segmentName','facility_name','user_id']]

In [165]:
validation = validation.set_index("id")
validation.columns

Index(['prob', 'Start_Time', 'Start_time_of_the_day', 'status', 'type',
       'prevStatus', 'areaName', 'segmentName', 'facility_name', 'user_id'],
      dtype='object')

In [166]:
validation.to_csv('pred_{}_Silver_Bullet.csv'.format(time))

In [167]:
validation#.head(30)

,prob,Start_Time,Start_time_of_the_day,status,type,prevStatus,areaName,segmentName,facility_name,user_id
id,,,,,,,,,,
199102,0.636987,2021-06-09,morning,confirmed,LPN,selected,Northeast Ohio,Senior Living,Avenue at Macedonia,10048
198788,0.555280,2021-06-09,morning,confirmed,LPN,selected,Northeast Ohio,Senior Living,Avenue at Macedonia,16041
199129,0.490924,2021-06-09,morning,confirmed,LPN,selected,Northeast Ohio,Senior Living,Avenue at Broadview Heights,19664
198805,0.572149,2021-06-09,morning,confirmed,STNA,selected,Northeast Ohio,Senior Living,Avenue at Broadview Heights,16536
197341,0.490933,2021-06-09,morning,confirmed,STNA,selected,Northeast Ohio,Senior Living,Avenue at North Ridgeville,17061
...,...,...,...,...,...,...,...,...,...,...
191649,0.104154,2021-07-10,morning,confirmed,RN,selected,Houston,Healthcare,Woodlands Specialty Hospital,7894
192531,0.772450,2021-07-10,afternoon,confirmed,STNA,selected,Northeast Ohio,Senior Living,CareCore at Willowood,9428
192145,0.447694,2021-07-11,afternoon,confirmed,STNA,selected,Northeast Ohio,Senior Living,CareCore at Willowood,4733


In [168]:
# select only northeast ohio and stna, and make a pivot table
# create a column called count
pivot_table = validation[(validation['areaName'] == 'Northeast Ohio') & 
           (validation['type'] == 'STNA') & (validation['prob'] > 0.55)].groupby(["Start_Time",
                                    "Start_time_of_the_day"]).size().reset_index(name='count').set_index("Start_Time")


In [169]:
# check if the count is above limiter of 5
# pivot_table['above_limiter'] = pivot_table.apply(lambda row: 2 if row['count'] >= 5 else 0, axis =1)

In [170]:
pivot_table.head(20)

,Start_time_of_the_day,count
Start_Time,,
2021-06-09,afternoon,4
2021-06-09,morning,8
2021-06-09,overnight,4
2021-06-10,afternoon,5
2021-06-10,morning,5
2021-06-10,overnight,4
2021-06-11,afternoon,2
2021-06-11,morning,8
2021-06-11,overnight,5


In [171]:
pivot_table.to_excel("plan2_pred_{}.xlsx".format(time))